In [ ]:
#Installing libraries if necessary
!pip install thefuzz nameparser spacy
!python -m spacy download pt_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 52.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#Library for data manipulation and analysis
import pandas as pd

#Standard library for working with regular expressions
import re

#Importing the fuzz and process functions - this library is used to compare and match strings based on similarity
from thefuzz import fuzz, process

#Imports the HumanName class from the nameparser library
from nameparser import HumanName

#Natural language processing (NLP) library
import spacy

In [ ]:
##Load the spaCy Portuguese language model
nlp = spacy.load("pt_core_news_sm")

In [ ]:
#Mounting Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading test data
df = pd.read_excel(
    "/content/drive/MyDrive/Colab Notebooks/collector_names.xls"
)

In [ ]:
#Confirming the dimensions of the dataset
print(df.shape)

(184, 2)


In [ ]:
#Checking the top 20
df.head(20)

,number,collector
0,1,A.Costa. G. Martinelli. Leitman. E.C. Dalcin. ...
1,2,André M. de Carvalho & G. Martinelli
2,3,g mARTINELLI
3,4,G.martinelli
4,5,G. martinelli
5,6,GMartinelli
6,7,G Martinelli
7,8,G .Martinelli
8,9,G . Martinelli
9,10,G Martinelli


In [ ]:
#Function to create a mapping dictionary based on the desired format
def create_mapping_dictionary(
    dataframe, column, keyword, target_name, threshold=80
):
    """
    Creates a mapping dictionary to standardize collector names containing a specific keyword.
      :param dataframe: DataFrame containing the collector names.
      :param column: Name of the column containing the collector names.
      :param keyword: Keyword to identify the collector to be standardized (e.g., "Forzza").
      :param target_name: Desired standardized name (e.g., "R.C. Forzza").
      :param threshold: Minimum similarity to consider matches (default = 80).
      :return: Mapping dictionary.
    """
    unique_names = dataframe[column].dropna().unique()
    mapping_dict = {}
    for name in unique_names:
        # Extract the first collector
        collectors = re.split(r'\s+(?:e|&|;)\s+', name)
        first_collector = collectors[0].strip().lower()

        # Check if the keyword is in the first collector
        if keyword.lower() in first_collector:
            mapping_dict[name] = target_name
    return mapping_dict

In [ ]:
#Function to standardize collector names
def standardize_collector_name(name, mapping, threshold = 80):
    """
    Standardizes the collector name using a mapping dictionary and fuzzy matching.
      :param name: Original collector name.
      :param mapping: Mapping dictionary for standardization.
      :param threshold: Minimum similarity to consider matches (default = 80).
      :return: Standardized collector name.
    """
    # Extract the first collector
    collectors = re.split(r'\s+(?:e|&|;)\s+', name)
    first_collector = collectors[0].strip()
    first_collector_lower = first_collector.lower()

    # Direct check in the mapping
    if first_collector_lower in mapping:
        return mapping[first_collector_lower]

    # If not found, use fuzzy matching
    best_match, score = process.extractOne(
        first_collector_lower,
        list(mapping.keys()),
        scorer=fuzz.token_sort_ratio
    )
    if score >= threshold:
        return mapping[best_match]
    else:
        # Fallback with nameparser
        parsed_name = HumanName(first_collector)
        return f"{parsed_name.first} {parsed_name.last}".strip()

In [ ]:
#Example usage - specify the keyword to search for and the desired standardized name
#keyword = "Forzza"
#target_name = "R.C. Forzza"
keyword = "Martinelli"
target_name = "G. Martinelli"

In [ ]:
#Create the mapping dictionary
mapping_dict = create_mapping_dictionary(df, 'collector', keyword, target_name)

In [ ]:
#Invert the mapping dictionary for easier lookup
inverted_mapping = {var.lower(): standard for var, standard in mapping_dict.items()}

In [ ]:
#Apply the standardization function to the DataFrame
df['standardized_name']=df['collector'].apply(lambda x: standardize_collector_name(x, inverted_mapping))

In [ ]:
# Display the results
df

,number,collector,standardized_name
0,1,A.Costa. G. Martinelli. Leitman. E.C. Dalcin. ...,G. Martinelli
1,2,André M. de Carvalho & G. Martinelli,André de Carvalho
2,3,g mARTINELLI,G. Martinelli
3,4,G.martinelli,G. Martinelli
4,5,G. martinelli,G. Martinelli
...,...,...,...
179,180,R. Mello-Silva & R. C. Forzza,R. Mello-Silva
180,181,R. Mello-Silva & R.C. Forzza,R. Mello-Silva
181,182,R. Mello-Silva,R. Mello-Silva
182,183,"R. Mello-Silva, R. C. Forzza, L. G. Temponi,",R. R. Mello-Silva


In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/padronizado.csv", index=False)